# Satria Data 2024

# Model & Data

In [ ]:
pip install gensim


In [ ]:
pip install Sastrawi


In [ ]:
!pip install --upgrade catboost


In [ ]:
!nvcc --version


In [ ]:
pip install sentence-transformers

In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, recall_score, balanced_accuracy_score
from transformers import AutoTokenizer, AutoModel
import torch
from catboost import CatBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from sklearn.decomposition import LatentDirichletAllocation
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from gensim.models import FastText
import numpy as np
import gensim.downloader as api
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Membuat stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('indonesian'))

In [ ]:
nltk.download('punkt')

In [ ]:
labeled_data =  pd.read_csv('/content/dataset_penyisihan_bdc_2024.csv',delimiter=";")
unlabeled_data = pd.read_csv('/content/dataset_unlabeled_penyisihan_bdc_2024.csv',delimiter=";")

In [ ]:
labeled_data

Preprocessing

In [ ]:
# Daftar kata yang akan dihapus
keywords_to_remove = ['anies', 'ganjar', 'baswedan', 'mahfud', 'md', 'prabowo', 'gibran']

def clean_text(text):
    text = re.sub(r'\[.*?\]', '', text)  # Hapus teks dalam kurung siku beserta kurung sikunya
    # Hapus RT atau RE di awal teks
    text = re.sub(r'^(RT|RE)\s+', '', text)
    # Hapus teks di belakang karakter @
    text = re.sub(r'@\S+', '', text)
    # Hapus URL
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Hapus karakter khusus dan angka
    text = re.sub(r'\d+', '', text)
    # Hapus karakter ASCII yang tidak diinginkan
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    # Hapus tanda baca
    text = re.sub(r'[^\w\s]', '', text)
    # Hapus spasi ekstra
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\byg\b', '', text, flags=re.IGNORECASE)  # Hapus kata RE
    # Pisahkan teks menjadi kata-kata individu
    words = text.split()
    # Hapus kata-kata tertentu
    filtered_words = [word for word in words if word.lower() not in keywords_to_remove]
    # Hapus kata imbuhan dan kata-kata stop_words jika ada
    filtered_words = [word for word in filtered_words if word.lower() not in stop_words]
    # Gabungkan kembali kata-kata yang telah difilter
    text = ' '.join(filtered_words)
    return text


In [ ]:
# Terapkan fungsi clean_text ke kolom teks
labeled_data['cleaned_text'] = labeled_data['text'].apply(clean_text)
unlabeled_data['cleaned_text'] = unlabeled_data['Text'].apply(clean_text)

# Tampilkan beberapa baris untuk memastikan teks telah dibersihkan
print(labeled_data[['text', 'cleaned_text']])
print(unlabeled_data[['Text', 'cleaned_text']])

In [ ]:
# Menghapus duplikat di labeled_data
labeled_data_cleaned = labeled_data.drop_duplicates()

# Menghapus duplikat di unlabeled_data
unlabeled_data_cleaned = unlabeled_data.drop_duplicates()


# Pisahkan text berdasarkan label

In [ ]:
# Pisahkan teks berdasarkan label
texts_by_label = labeled_data.groupby('label')['cleaned_text'].apply(lambda x: ' '.join(x)).reset_index()

In [ ]:
def get_top_n_words(text, n=10):
    words = text.split()
    word_counts = Counter(words)
    return word_counts.most_common(n)

# Hitung frekuensi kata untuk setiap label
texts_by_label['top_words'] = texts_by_label['cleaned_text'].apply(lambda x: get_top_n_words(x, n=10))

# Tampilkan hasil
print(texts_by_label[['label', 'top_words']])


In [ ]:
# Visualisasi frekuensi kata teratas untuk setiap label
def plot_top_words(top_words, label):
    words, counts = zip(*top_words)
    plt.figure(figsize=(10, 6))
    sns.barplot(x=list(counts), y=list(words))
    plt.title(f"Top Words in {label}")
    plt.xlabel("Counts")
    plt.ylabel("Words")
    plt.show()

# Plot untuk setiap label
for index, row in texts_by_label.iterrows():
    plot_top_words(row['top_words'], row['label'])


## Fitur Ekstraksi

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, classification_report
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
from transformers import BertTokenizer, TFBertModel
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from sklearn.metrics import balanced_accuracy_score, classification_report
from transformers.data.processors.utils import InputExample

# Updated keyword dictionary
keyword_dict = {
    'Ekonomi': ['jnk', 'ekonomi', 'tani'],
    'Demografi': ['kalangan muda', 'kalangan'],
    'Geografi': ['pulau', 'jakarta', 'kota'],
    'Ideologi': ['dasar negara', 'dasar', 'negara', 'konstitusi'],
    'Pertahanan dan Keamanan': ['tahan', 'tni', 'serang', 'akun', 'masuk', 'rawan'],
    'Politik': ['ganjarmahfudpilihanumat', 'dukung'],
    'Sumber Daya Alam': ['air', 'tani', 'indosentris']
}

# Fungsi untuk mengonversi data teks menjadi fitur LDA
def convert_text_to_lda_features(texts, keyword_dict):
    vectorizer = CountVectorizer(max_features=1000, stop_words='english')
    X = vectorizer.fit_transform(texts)

    lda = LatentDirichletAllocation(n_components=10, random_state=42)
    lda.fit(X)

    lda_features = lda.transform(X)

    # Tambahkan fitur kata kunci
    keyword_features = np.zeros((len(texts), len(keyword_dict)))
    for i, text in enumerate(texts):
        for j, (category, keywords) in enumerate(keyword_dict.items()):
            if any(keyword in text for keyword in keywords):
                keyword_features[i, j] = 1

    combined_features = np.hstack((lda_features, keyword_features))
    return combined_features

# Fungsi untuk mengonversi data menjadi InputExample untuk BERT
def convert_data_to_examples(texts, labels):
    return [InputExample(guid=None, text_a=text, text_b=None, label=label) for text, label in zip(texts, labels)]

# Fungsi untuk mengonversi InputExample menjadi tf.data.Dataset
def convert_examples_to_tf_dataset(examples, tokenizer, lda_features, max_length=128):
    features = []

    for e, lda_feat in zip(examples, lda_features):
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            return_token_type_ids=True,
            return_attention_mask=True,
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"], input_dict["token_type_ids"], input_dict["attention_mask"])

        # Konversi fitur LDA menjadi tf.Tensor
        lda_tensor = tf.convert_to_tensor(lda_feat, dtype=tf.float32)

        # Gabungkan input_ids, token_type_ids, attention_mask, dan lda_tensor
        combined_input = {
            "input_ids": tf.convert_to_tensor(input_ids, dtype=tf.int32),
            "token_type_ids": tf.convert_to_tensor(token_type_ids, dtype=tf.int32),
            "attention_mask": tf.convert_to_tensor(attention_mask, dtype=tf.int32),
            "lda_features": lda_tensor
        }

        features.append((combined_input, e.label))

    def gen():
        for f in features:
            yield f

    return tf.data.Dataset.from_generator(
        gen,
        output_signature=(
            {
                "input_ids": tf.TensorSpec(shape=(max_length,), dtype=tf.int32),
                "token_type_ids": tf.TensorSpec(shape=(max_length,), dtype=tf.int32),
                "attention_mask": tf.TensorSpec(shape=(max_length,), dtype=tf.int32),
                "lda_features": tf.TensorSpec(shape=(20,), dtype=tf.float32)  # 10 LDA + 10 keyword categories
            },
            tf.TensorSpec(shape=(), dtype=tf.int64)
        )
    )

# Pisahkan data menjadi fitur (X) dan label (y)
X = labeled_data['cleaned_text']
y = labeled_data['label']

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Bagi data menjadi set pelatihan dan set pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-large-p2")

# Convert texts to LDA features
train_lda_features = convert_text_to_lda_features(X_train, keyword_dict)
test_lda_features = convert_text_to_lda_features(X_test, keyword_dict)

# Convert LDA features to InputExamples
train_InputExamples = convert_data_to_examples(X_train, y_train)
validation_InputExamples = convert_data_to_examples(X_test, y_test)

# Convert examples to tf.data.Dataset with LDA features
train_data = convert_examples_to_tf_dataset(train_InputExamples, tokenizer, train_lda_features)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(validation_InputExamples, tokenizer, test_lda_features)
validation_data = validation_data.batch(32)

# Custom model combining BERT and LDA features
class CustomBertModel(tf.keras.Model):
    def __init__(self, bert_model, num_labels):
        super(CustomBertModel, self).__init__()
        self.bert = bert_model
        self.dense = tf.keras.layers.Dense(num_labels, activation='softmax')
        self.dropout = tf.keras.layers.Dropout(0.1)
        self.lda_dense = tf.keras.layers.Dense(768, activation='relu')

    def call(self, inputs, training=False):
        bert_inputs = {k: v for k, v in inputs.items() if k != 'lda_features'}
        lda_features = inputs['lda_features']

        bert_output = self.bert(bert_inputs, training=training)[1]
        lda_output = self.lda_dense(lda_features)

        combined_output = tf.concat([bert_output, lda_output], axis=-1)
        combined_output = self.dropout(combined_output, training=training)
        return self.dense(combined_output)

# Load pre-trained BERT model
bert_model = TFBertModel.from_pretrained("indobenchmark/indobert-large-p2")

# Create custom model
custom_model = CustomBertModel(bert_model, num_labels=len(label_encoder.classes_))

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

custom_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Train the model
history = custom_model.fit(train_data, epochs=2, validation_data=validation_data)

# Evaluate the model
preds = custom_model.predict(validation_data)
y_pred = np.argmax(preds, axis=1)

# Decode labels
y_test_decoded = label_encoder.inverse_transform(y_test)
y_pred_decoded = label_encoder.inverse_transform(y_pred)

print("Balanced Accuracy:", balanced_accuracy_score(y_test_decoded, y_pred_decoded))
print("Classification Report:\n", classification_report(y_test_decoded, y_pred_decoded))


In [ ]:
from transformers import BertTokenizer, TFBertModel,TFBertForSequenceClassification
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
# Fungsi untuk mengonversi data teks menjadi fitur LDA
def convert_text_to_lda_features(texts):
    vectorizer = CountVectorizer(max_features=1000, stop_words='english')
    X = vectorizer.fit_transform(texts)

    lda = LatentDirichletAllocation(n_components=10, random_state=42)
    lda.fit(X)

    return lda.transform(X)

# Fungsi untuk mengonversi data menjadi InputExample untuk BERT
def convert_data_to_examples(texts, labels):
    return [InputExample(guid=None, text_a=text, text_b=None, label=label) for text, label in zip(texts, labels)]

# Fungsi untuk mengonversi InputExample menjadi tf.data.Dataset
def convert_examples_to_tf_dataset(examples, tokenizer, lda_features, max_length=128):
    features = []

    for e, lda_feat in zip(examples, lda_features):
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            return_token_type_ids=True,
            return_attention_mask=True,
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"], input_dict["token_type_ids"], input_dict["attention_mask"])

        # Konversi fitur LDA menjadi tf.Tensor
        lda_tensor = tf.convert_to_tensor(lda_feat, dtype=tf.float32)

        # Gabungkan input_ids, token_type_ids, attention_mask, dan lda_tensor
        combined_input = {
            "input_ids": input_ids,
            "token_type_ids": token_type_ids,
            "attention_mask": attention_mask,
            "lda_features": lda_tensor
        }

        features.append((combined_input, e.label))

    def gen():
        for f in features:
            yield f

    return tf.data.Dataset.from_generator(
        gen,
        output_signature=(
            {
                "input_ids": tf.TensorSpec(shape=(None,), dtype=tf.int32),
                "token_type_ids": tf.TensorSpec(shape=(None,), dtype=tf.int32),
                "attention_mask": tf.TensorSpec(shape=(None,), dtype=tf.int32),
                "lda_features": tf.TensorSpec(shape=(10,), dtype=tf.float32)
            },
            tf.TensorSpec(shape=(), dtype=tf.int64)
        )
    )



# Pisahkan data menjadi fitur (X) dan label (y)
X = labeled_data['cleaned_text']
y = labeled_data['label']

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Bagi data menjadi set pelatihan dan set pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-large-p2")

# Convert texts to LDA features
train_lda_features = convert_text_to_lda_features(X_train)
test_lda_features = convert_text_to_lda_features(X_test)

# Convert LDA features to InputExamples
train_InputExamples = convert_data_to_examples(X_train, y_train)
validation_InputExamples = convert_data_to_examples(X_test, y_test)

# Convert examples to tf.data.Dataset with LDA features
train_data = convert_examples_to_tf_dataset(train_InputExamples, tokenizer, train_lda_features)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(validation_InputExamples, tokenizer, test_lda_features)
validation_data = validation_data.batch(32)

# Custom model combining BERT and LDA features
class CustomBertModel(tf.keras.Model):
    def __init__(self, bert_model, num_labels):
        super(CustomBertModel, self).__init__()
        self.bert = bert_model
        self.dense = tf.keras.layers.Dense(num_labels, activation='softmax')
        self.dropout = tf.keras.layers.Dropout(0.1)
        self.lda_dense = tf.keras.layers.Dense(768, activation='relu')

    def call(self, inputs, training=False):
        bert_inputs = {k: v for k, v in inputs.items() if k != 'lda_features'}
        lda_features = inputs['lda_features']

        bert_output = self.bert(bert_inputs, training=training)[1]
        lda_output = self.lda_dense(lda_features)

        combined_output = tf.concat([bert_output, lda_output], axis=-1)
        combined_output = self.dropout(combined_output, training=training)
        return self.dense(combined_output)

# Load pre-trained BERT model
bert_model = TFBertModel.from_pretrained("indobenchmark/indobert-large-p2")

# Create custom model
custom_model = CustomBertModel(bert_model, num_labels=len(label_encoder.classes_))

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

custom_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Train the model
history = custom_model.fit(train_data, epochs=2, validation_data=validation_data)

# Evaluate the model
preds = custom_model.predict(validation_data)
y_pred = np.argmax(preds, axis=1)

# Decode labels
y_test_decoded = label_encoder.inverse_transform(y_test)
y_pred_decoded = label_encoder.inverse_transform(y_pred)

print("Balanced Accuracy:", balanced_accuracy_score(y_test_decoded, y_pred_decoded))
print("Classification Report:\n", classification_report(y_test_decoded, y_pred_decoded))

In [ ]:
# Pisahkan data menjadi fitur (X) dan label (y)
X = labeled_data['cleaned_text']
y = labeled_data['label']

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Verifikasi bahwa semua label telah dikonversi ke numerik
print("Encoded labels:", set(y_encoded))

# Bagi data menjadi set pelatihan dan set pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-large-p2")

# Fungsi untuk membuat data input untuk BERT
def convert_data_to_examples(texts, labels):
    return [InputExample(guid=None, text_a=text, text_b=None, label=label) for text, label in zip(texts, labels)]

# Konversi data ke InputExample
train_InputExamples = convert_data_to_examples(X_train, y_train)
validation_InputExamples = convert_data_to_examples(X_test, y_test)

# Fungsi untuk membuat InputFeatures
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = []

    for e in examples:
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length,
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True,
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"], input_dict["token_type_ids"], input_dict["attention_mask"])

        features.append(
            InputFeatures(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label)
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        (
            {
                "input_ids": tf.int32,
                "attention_mask": tf.int32,
                "token_type_ids": tf.int32,
            },
            tf.int64,
        ),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

# Konversi data ke tf.data.Dataset
train_data = convert_examples_to_tf_dataset(train_InputExamples, tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(validation_InputExamples, tokenizer)
validation_data = validation_data.batch(32)

# Load model BERT untuk klasifikasi
model = TFBertForSequenceClassification.from_pretrained("indobenchmark/indobert-large-p2", num_labels=len(label_encoder.classes_))

# Kompilasi model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Latih model
model.fit(train_data, epochs=2, validation_data=validation_data)

# Evaluasi model
preds = model.predict(validation_data)
y_pred = np.argmax(preds.logits, axis=1)

print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))

In [ ]:
# Konversi teks ke format input untuk BERT
unlabeled_examples = convert_data_to_examples(unlabeled_data['cleaned_text'], [0]*len(unlabeled_data))
unlabeled_data_tf = convert_examples_to_tf_dataset(unlabeled_examples, tokenizer)
unlabeled_data_tf = unlabeled_data_tf.batch(32)

# Prediksi label untuk data tidak berlabel
unlabeled_preds = model.predict(unlabeled_data_tf)
unlabeled_y_pred = np.argmax(unlabeled_preds.logits, axis=1)

# Konversi label numerik kembali ke label asli
unlabeled_labels = label_encoder.inverse_transform(unlabeled_y_pred)

# Simpan hasil prediksi ke file CSV untuk submission
submission = pd.DataFrame({'IDText': unlabeled_data['IDText'], 'Kelas': unlabeled_labels})
submission.to_csv('submission.csv', index=False)

print("Submission file created successfully!")

In [ ]:
 # Pisahkan data menjadi fitur (X) dan label (y)
X = labeled_data['cleaned_text']
y = labeled_data['label']

# Bagi data menjadi set pelatihan dan set pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train FastText model
sentences = [text.split() for text in X_train]
fasttext_model = FastText(sentences, vector_size=100, window=5, min_count=1, workers=4, sg=1)

def get_fasttext_embeddings(text, model):
    words = text.split()
    embeddings = [model.wv[word] for word in words if word in model.wv]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(model.vector_size)

# Compute FastText embeddings for training and test set
X_train_fasttext = np.array([get_fasttext_embeddings(text, fasttext_model) for text in X_train])
X_test_fasttext = np.array([get_fasttext_embeddings(text, fasttext_model) for text in X_test])

# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer(max_features=10000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Apply LDA to get topic distributions
n_topics = 10
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
X_train_lda = lda.fit_transform(X_train_tfidf)
X_test_lda = lda.transform(X_test_tfidf)

# Load IndoBERT model from sentence-transformers
model = SentenceTransformer("indobenchmark/indobert-large-p2")

# Compute IndoBERT embeddings for training set
train_sentence_embeddings = model.encode(X_train.tolist(), show_progress_bar=True)

# Compute IndoBERT embeddings for test set
test_sentence_embeddings = model.encode(X_test.tolist(), show_progress_bar=True)

# Combine LDA features, IndoBERT embeddings, and FastText embeddings
X_train_combined = np.hstack((X_train_lda, train_sentence_embeddings, X_train_fasttext))
X_test_combined = np.hstack((X_test_lda, test_sentence_embeddings, X_test_fasttext))

# Train CatBoost model using combined features
model = CatBoostClassifier(iterations=1000, task_type="GPU", devices='0:1', verbose=0)
model.fit(X_train_combined, y_train)

# Predict on test set using combined features
y_pred = model.predict(X_test_combined)

# Evaluate model performance
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
#Pisahkan data menjadi fitur (X) dan label (y)
X = labeled_data['cleaned_text']
y = labeled_data['label']

# Bagi data menjadi set pelatihan dan set pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer(max_features=10000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Apply LDA to get topic distributions
n_topics = 10
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
X_train_lda = lda.fit_transform(X_train_tfidf)
X_test_lda = lda.transform(X_test_tfidf)

# Load IndoBERT model from sentence-transformers
model = SentenceTransformer("indobenchmark/indobert-large-p2")

# Compute IndoBERT embeddings for training set
train_sentence_embeddings = model.encode(X_train.tolist(), show_progress_bar=True)

# Compute IndoBERT embeddings for test set
test_sentence_embeddings = model.encode(X_test.tolist(), show_progress_bar=True)

# Ensure IndoBERT embeddings are 2D by averaging word embeddings in each sentence
train_sentence_embeddings_2d = np.array([np.mean(emb, axis=0) if len(emb.shape) == 2 else emb for emb in train_sentence_embeddings])
test_sentence_embeddings_2d = np.array([np.mean(emb, axis=0) if len(emb.shape) == 2 else emb for emb in test_sentence_embeddings])

# Check dimensions
print("X_train_lda shape:", X_train_lda.shape)
print("train_sentence_embeddings_2d shape:", train_sentence_embeddings_2d.shape)
print("X_test_lda shape:", X_test_lda.shape)
print("test_sentence_embeddings_2d shape:", test_sentence_embeddings_2d.shape)

# Combine LDA features and IndoBERT embeddings
X_train_combined = np.hstack((X_train_lda, train_sentence_embeddings_2d))
X_test_combined = np.hstack((X_test_lda, test_sentence_embeddings_2d))

# Train CatBoost model using combined features
catboost_model = CatBoostClassifier(iterations=1000, task_type="GPU", devices='0:1', verbose=0)
catboost_model.fit(X_train_combined, y_train)

# Predict on test set using combined features
y_pred = catboost_model.predict(X_test_combined)

# Evaluate model performance
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
 # Pisahkan data menjadi fitur (X) dan label (y)
X = labeled_data['cleaned_text']
y = labeled_data['label']
# Bagi data menjadi set pelatihan dan set pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer(max_features=10000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Apply LDA to get topic distributions
n_topics = 10
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
X_train_lda = lda.fit_transform(X_train_tfidf)
X_test_lda = lda.transform(X_test_tfidf)

# Load IndoBERT model from sentence-transformers
model = SentenceTransformer("indobenchmark/indobert-large-p2")

# Compute IndoBERT embeddings for training set
train_sentence_embeddings = model.encode(X_train.tolist(), show_progress_bar=True)

# Compute IndoBERT embeddings for test set
test_sentence_embeddings = model.encode(X_test.tolist(), show_progress_bar=True)

# Combine LDA features and IndoBERT embeddings
import numpy as np
X_train_combined = np.hstack((X_train_lda, train_sentence_embeddings))
X_test_combined = np.hstack((X_test_lda, test_sentence_embeddings))

# Train CatBoost model using combined features
model = CatBoostClassifier(iterations=1000, task_type="GPU", devices='0:1', verbose=0)
model.fit(X_train_combined, y_train)

# Predict on test set using combined features
y_pred = model.predict(X_test_combined)

# Evaluate model performance
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# Inisialisasi TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=100, stop_words=stopwords.words('indonesian'))

# Fit dan transform teks
tfidf_matrix = tfidf_vectorizer.fit_transform(texts_by_label['cleaned_text'])

# Membuat DataFrame dari hasil TF-IDF
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
tfidf_df['label'] = texts_by_label['label']

# Tampilkan fitur TF-IDF untuk setiap label
print(tfidf_df)

In [ ]:
 # Apply LDA to get topic distributions
n_topics = 10
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
X_train_lda = lda.fit_transform(X_train_tfidf)
X_test_lda = lda.transform(X_test_tfidf)

# Mencoba Model

In [ ]:
# Pisahkan data menjadi fitur (X) dan label (y)
X = labeled_data['cleaned_text']
y = labeled_data['label']

In [ ]:
# Bagi data menjadi set pelatihan dan set pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Load tokenizer IndoBERT yang telah dituning ulang
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-large-p2")
model = AutoModel.from_pretrained("indobenchmark/indobert-large-p2")

In [ ]:
# Tokenisasi teks
X_train_tokenized = tokenizer(X_train.tolist(), padding=True, truncation=True, return_tensors="pt")
X_test_tokenized = tokenizer(X_test.tolist(), padding=True, truncation=True, return_tensors="pt")

# Konversi token menjadi teks
X_train_text = tokenizer.batch_decode(X_train_tokenized['input_ids'], skip_special_tokens=True)
X_test_text = tokenizer.batch_decode(X_test_tokenized['input_ids'], skip_special_tokens=True)

# Ekstraksi fitur dengan TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_text)
X_test_tfidf = tfidf_vectorizer.transform(X_test_text)

In [ ]:
# Buat pipeline dengan Logistic Regression
pipeline = Pipeline([
    ('clf', CatBoostClassifier(iterations=1000, task_type="GPU", devices='0:1', verbose=0))
])

In [ ]:
# Latih model
pipeline.fit(X_train_tfidf, y_train)

# Prediksi pada set pengujian
y_pred = pipeline.predict(X_test_tfidf)

# Evaluasi kinerja model dengan macro recall
macro_recall = recall_score(y_test, y_pred, average='macro')

# Print macro recall
print("Macro Recall:", macro_recall)

In [ ]:
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred))